In [1]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [2]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

 ········


In [2]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [4]:
config = {"configurable": {"session_id": "abc2"}}

In [6]:
from langchain_core.messages import HumanMessage

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Moiz")],
    config=config
)

response.content

Parent run 208186a1-3b40-46bf-9a0c-31baaef6c647 not found for run 0e4a259d-0ba4-40b8-a1ad-d5b9a24f38e5. Treating as a root run.


'Hello Moiz! How can I assist you today?'

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response.content

Parent run 28cc5d44-5d08-4088-a63b-6b965d2748c9 not found for run 2f231ab5-c141-4ba6-956c-a8b786adb4af. Treating as a root run.


'Your name is Moiz. How can I help you?'

In [8]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response.content

Parent run 9d4a0d19-deb5-41ae-bfa4-38e1bbd7c90d not found for run c3bca508-5960-49b8-b74a-39b2e4d71a8e. Treating as a root run.


"I'm sorry, I don't know your name. Can you please tell me?"

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [12]:
response = chain.invoke({"messages": [HumanMessage(content="Hi! I'm Moiz")]})

response.content

'Hello Moiz! How can I assist you today?'

In [13]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [14]:
config = {"configurable": {"session_id": "abc5"}}

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Moiz")],
    config=config
)

response.content

Parent run c456edc8-8d43-4298-9b1d-7bd7c547541b not found for run 749535fa-6575-4ee8-bcff-48faebecbc8b. Treating as a root run.


'Hello Moiz! How can I assist you today?'

In [16]:
response = chain.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Moiz")], "language": "Spanish"}
)

response.content

'Hello Moiz! How can I assist you today?'

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm Moiz")], "language": "Spanish"}
)

response.content

'¡Hola, Moiz! ¿En qué puedo ayudarte hoy?'

In [19]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [21]:
from langchain_core.messages import AIMessage

messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [24]:
config = {"configurable": {"session_id": "abc20"}}

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream?")],
        "language": "English"
    },
    config=config
)

response.content

'Your favorite ice cream flavor is vanilla.'

In [28]:
config = {"configurable": {"session_id": "abc15"}}

for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="Hi! I'm Moiz. Tell me a joke")],
        "language": "English"
    },
    config=config
):
    print(r.content, end="|")

KeyError: 'input'